# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [33]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df
df.action.value_counts()


view     6328
click    1860
Name: action, dtype: int64

In [34]:
clickers = set(df[df.action == 'click']['id'].unique())
viewers = set(df[df.action == 'view']['id'].unique())
clicked_not_viewed_count = len(clickers - viewers)
viewed_not_clicked_count = len(viewers - clickers)
print('Number of individuals who clicked but did not view:', clicked_not_viewed_count)
print('Number of individuals who viewed but did not click:', viewed_not_clicked_count)
print('Viewers are {} and clickers {}'.format(len(viewers), len(clickers)))


Number of individuals who clicked but did not view: 0
Number of individuals who viewed but did not click: 4468
Viewers are 6328 and clickers 1860


In [35]:
control_group = set(df[df['group'] == 'control']['id'])
experiment_group = set(df[df['group'] == 'experiment']['id'])
overlap = control_group.intersection(experiment_group)
print('Number of overlapping individuals:', len(overlap))

Number of overlapping individuals: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [36]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [37]:
#Your code here
#Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [38]:
# Assuming 'group' is the column representing the groups ('control', 'experiment')
control_sample_sizes = df[df['group'] == 'control']['id'].value_counts()
experiment_sample_sizes = df[df['group'] == 'experiment']['id'].value_counts()

print("Sample_size of control: {}\tSample_size of experiments: {}".format(len(control_sample_sizes), len(experiment_sample_sizes)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()


Sample_size of control: 3332	Sample_size of experiments: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [39]:
import numpy as np
import scipy.stats as stats

def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """

    numerator = a.mean() - b.mean()
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. This function returns the degrees of freedom """
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


def p_value_welch_ttest(a, b, two_sided=False):
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [40]:
p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [41]:
#Your code here
control_clicks = control.click.mean()
experiment_clicks = control_clicks * len(experiment)
print(experiment_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [42]:
#Your code here
actual_clicks_experiment = experiment.click.sum()  # Actual number of clicks in the experiment group (replace with the actual value)

# Calculate the standard deviation of the actual number of clicks in the experiment group
n = len(experiment)
p = control_clicks
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

# Calculate the number of standard deviations
z_score = (actual_clicks_experiment - experiment_clicks) / std

print("Number of Standard Deviations from the Estimate:", z_score)


24.568547907005815
Number of Standard Deviations from the Estimate: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [43]:
#Your code here
import scipy.stats as stats
p_value = 1 - stats.norm.cdf(z_score)
print(p_value)

0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [44]:
'''
Yes they match because they both lead to the rejection of the null hypothesis.
'''

'\nYes they match because they both lead to the rejection of the null hypothesis.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.